In [ ]:
import sys
sys.path.append("./../")

In [ ]:
import torch
import torch.nn as nn
import torchvision
#
from effcn.layers import PrimaryCaps

In [ ]:
class PrimaryCaps(nn.Module):
    def __init__(self, num_capsules=8, in_channels=256, out_channels=32, kernel_size=9, num_routes=32 * 6 * 6):
        super(PrimaryCaps, self).__init__()
        self.num_routes = num_routes
        self.capsules = nn.ModuleList([
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=1, padding=0, bias=False)
            for _ in range(num_capsules)])

    def forward(self, x):
        u = [capsule(x) for capsule in self.capsules]
        #u = torch.stack(u, dim=1)
        #u = u.view(x.size(0), self.num_routes, -1)
        #u = self.squash(u)
        return u
        #return self.squash(u)

    def squash(self, input_tensor):
        squared_norm = (input_tensor ** 2).sum(-1, keepdim=True)
        output_tensor = squared_norm * input_tensor / ((1. + squared_norm) * torch.sqrt(squared_norm))
        return output_tensor

In [ ]:
model = nn.Conv2d(3, 10, kernel_size=(2, 2),stride=1, bias=False)
#
x = torch.rand((1, 3, 4, 4))
y = model(x)
print(y.shape)

In [ ]:
filters = list(model.parameters())[0]
print(filters.shape)

In [ ]:
backbone = nn.Sequential(
    nn.Conv2d(1, 256, kernel_size=(9, 9)),
    nn.ReLU(inplace=True),
    nn.Conv2d(256, 32, kernel_size=(9, 9), stride=2),
)
primecaps = PrimaryCaps()

In [ ]:
x = torch.rand((1, 1, 28, 28))
x_bb = backbone(x)
print(x_bb.shape)

In [ ]:
x_bb = layers(x)
print(x_bb.shape)
x_pc = primecaps(x_bb)
#print(x_pc.shape)

In [ ]:
print(len(x_pc))
print(x_pc[0].shape)

In [ ]:
weights = list(primecaps.capsules[0].parameters())[0]

In [ ]:
weights.shape

In [ ]:
def capsnet_graph(input_shape, routing):
    """
    Original CapsNet graph architecture described in "dynamic routinig between capsules".
    
    Parameters
    ----------   
    input_shape: list
        network input shape
    routing: int
        number of routing iterations
    """
    inputs = tf.keras.Input(input_shape)
    
    x = tf.keras.layers.Conv2D(256, 9, activation="relu")(inputs)
    primary = PrimaryCaps(C=32, L=8, k=9, s=2)(x)
    digit_caps = DigitCaps(10, 16, routing=routing)(primary)  
    digit_caps_len = Length(name='capsnet_output_len')(digit_caps)
    pr_shape = primary.shape
    primary = tf.reshape(primary,(-1,pr_shape[1]*pr_shape[2]*pr_shape[3],pr_shape[-1]))

    return tf.keras.Model(inputs=inputs,outputs=[primary, digit_caps, digit_caps_len] , name='Original_CapsNet')